In [3]:
import numpy as np
import requests
import shapefile
import geopandas as gpd

In [4]:
from io import BytesIO
from matplotlib.pyplot import imshow
from PIL import Image
from numpy.core.defchararray import strip

In [5]:
URL = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&REQUEST=GetMap&layers" \
      "=MODIS_Aqua_CorrectedReflectance_TrueColor&version=1.3.0&crs=EPSG:4326&transparent=false&width={}&height={" \
      "}&bbox={}&format=image/tiff&time={} "
KM_PER_DEG_AT_EQ = 111.

In [6]:
def calculate_width_height(extent, resolution):
    """
    extent: [lower_latitude, left_longitude, higher_latitude, right_longitude], EG: [51.46162974683544,
    -22.94768591772153,53.03698575949367,-20.952234968354432] resolution: represents the pixel resolution,
    i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5, 1, 5, 10]
    """
    lats = extent[::2]
    lons = extent[1::2]
    km_per_deg_at_lat = KM_PER_DEG_AT_EQ * np.cos(np.pi * np.mean(lats) / 180.)
    width = int((lons[1] - lons[0]) * km_per_deg_at_lat / resolution)
    height = int((lats[1] - lats[0]) * KM_PER_DEG_AT_EQ / resolution)
    print(width, height)
    return width, height

In [7]:
def modis_url(time, extent, resolution):
    """
    time: utc time in iso format EG: 2020-02-19T00:00:00Z extent: [lower_latitude, left_longitude, higher_latitude,
    right_longitude], EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432] resolution:
    represents the pixel resolution, i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5,
    1, 5, 10]
    """
    width, height = calculate_width_height(extent, resolution)
    extent = ','.join(map(lambda x: str(x), extent))
    return width, height, URL.format(width, height, extent, time)

In [8]:
import os
rootdir = '/Users/arianemaharaj/Documents/NASA/Data'
Dirname = [x[0] for x in os.walk(rootdir)]
Dirname.pop(0)
print (Dirname[0])
len(Dirname)
#207 useful directory names

/Users/arianemaharaj/Documents/NASA/Data/high-latitude-dust_2010-12-01_234


207

In [9]:
import re
temp = list()
for fname in Dirname:
    str0 = fname.split('_')
    str1 = str0[1]
    str2 = fname[fname.rindex('_'):]
    date = str1+str2
    #res = re.findall("high-latitude-dust_(\d+)", fname)
    #if not res: continue
    #print (res[0])
    temp.append (date)

In [10]:
len (temp)

207

In [11]:
temp.sort()
dates = temp
print (dates[1:10])
print (dates[-2:])
len(dates)

['2002-10-23_274', '2003-01-19_53', '2003-01-19_77', '2003-03-09_270', '2003-03-11_268', '2003-03-12_263', '2003-03-12_265', '2003-03-12_267', '2003-03-12_269']
['2019-11-09_289', 'Pics_Pics']


207

In [12]:
dates.pop(-1)
print (dates[1:5])
print (dates[-2:])
len(dates)

['2002-10-23_274', '2003-01-19_53', '2003-01-19_77', '2003-03-09_270']
['2019-03-26_111', '2019-11-09_289']


206

In [13]:
bbox = list()
for i in dates:
    sh = shapefile.Reader("../../Documents/NASA/Data/high-latitude-dust_" + i + "/high-latitude-dust_" + i)
    shapes = sh.shapes()
    box = shapes[0].bbox
    bbox.append(box)

In [14]:
bbox[1:10]

[[-19.05626, 62.637483, -17.143616, 63.602063],
 [-67.575581, -45.764561, -63.257844, -44.841727],
 [-67.54834, -45.711837, -63.500977, -44.850475],
 [-146.614203, 58.592352, -144.270666, 60.543486],
 [-146.467104, 59.275672, -144.136757, 60.569836],
 [-141.317987, 58.227593, -138.24017, 59.471221],
 [-141.622253, 58.297892, -139.206345, 59.383326],
 [-142.852868, 59.326216, -140.118007, 59.857944],
 [-146.700115, 59.392126, -144.765487, 60.556654]]

In [238]:
# sh = shapefile.Reader("../../Documents/NASA/high-latitude-dust_2012-12-04_221/high-latitude-dust_2012-12-04_221")
# shapes = sh.shapes()
# bbox = shapes[0].bbox
# bbox

In [239]:
# bboxt= [bbox[1],bbox[0],bbox[3],bbox[2]]
# bboxt

In [15]:
bbox_rotated = []
for i in range(len(bbox)):
    bbox_rotated.append([bbox[i][1],bbox[i][0],bbox[i][3],bbox[i][2]])
print (bbox_rotated[1:10])
print(bbox_rotated[-1])

[[62.637483, -19.05626, 63.602063, -17.143616], [-45.764561, -67.575581, -44.841727, -63.257844], [-45.711837, -67.54834, -44.850475, -63.500977], [58.592352, -146.614203, 60.543486, -144.270666], [59.275672, -146.467104, 60.569836, -144.136757], [58.227593, -141.317987, 59.471221, -138.24017], [58.297892, -141.622253, 59.383326, -139.206345], [59.326216, -142.852868, 59.857944, -140.118007], [59.392126, -146.700115, 60.556654, -144.765487]]
[57.059541, -144.166992, 59.976433, -138.023437]


In [16]:
# width, height, url = modis_url('2012-12-04T00:00:00Z',
#                                bboxt, 0.25)

In [17]:
dateonly = list()
for fname in Dirname:
    str0 = fname.split('_')
    dateonly.append (str0[1])

In [18]:
print (len(dateonly))
print (dateonly[1:10])

207
['2013-11-17', '2018-02-03', '2017-11-07', '2019-02-25', '2010-01-24', '2003-03-12', '2019-02-25', '2003-10-30', '2009-08-05']


In [19]:
dateonly.sort()
print (dateonly[1:10])
print (dateonly[-4:])

['2002-10-23', '2003-01-19', '2003-01-19', '2003-03-09', '2003-03-11', '2003-03-12', '2003-03-12', '2003-03-12', '2003-03-12']
['2019-02-25', '2019-03-26', '2019-11-09', 'Pics']


In [20]:
dateonly.pop(-1)
print (len(dateonly))
print (dateonly[-4:])

206
['2019-02-25', '2019-02-25', '2019-03-26', '2019-11-09']


In [246]:
#sh = shapefile.Reader("../../Documents/NASA/high-latitude-dust_7/high-latitude-dust_7")
# response = requests.get(strip(url))
# img = BytesIO(response.content)

In [247]:
# im = Image.open(img)
# im.show()
#Image.open(img)

In [255]:
#from django.core.files import File
import shutil 
for i, j, k in zip(dateonly, bbox_rotated, dates):
    try:
        width, height, url = modis_url(i+'T00:00:00Z',j, 0.25)
        response = requests.get(strip(url))
        img = BytesIO(response.content)
        im = Image.open(img)
        #im.show()
        im.save("../../Documents/NASA/Data/Dust_Pics/high-latitude-dust_"+k+".png")
        print("Saved file: " + "high-latitude-dust_"+k+".png")
    except:
        shutil.move("../../Documents/NASA/Data/high-latitude-dust_"+k, "../../Documents/NASA/No_Data")
        print("Error" + "high-latitude-dust_"+k)
   # im.qrcode_file.save('../../Documents/NASA/Data/Dust_Pics/high-latitude-dust'+k, File(im), save=False) 

153 211
Saved file: high-latitude-dust_2002-10-22_276.png
383 428
Saved file: high-latitude-dust_2002-10-23_274.png
1348 409
Saved file: high-latitude-dust_2003-01-19_53.png
1264 382
Saved file: high-latitude-dust_2003-01-19_77.png
527 866
Saved file: high-latitude-dust_2003-03-09_270.png
518 574
Saved file: high-latitude-dust_2003-03-11_268.png
706 552
Saved file: high-latitude-dust_2003-03-12_263.png
555 481
Saved file: high-latitude-dust_2003-03-12_265.png
614 236
Saved file: high-latitude-dust_2003-03-12_267.png
429 517
Saved file: high-latitude-dust_2003-03-12_269.png
178 179
Saved file: high-latitude-dust_2003-03-12_271.png
404 406
Saved file: high-latitude-dust_2003-03-13_146.png
485 419
Saved file: high-latitude-dust_2003-03-13_264.png
473 417
Saved file: high-latitude-dust_2003-03-13_266.png
341 376
Saved file: high-latitude-dust_2003-03-13_32.png
519 743
Saved file: high-latitude-dust_2003-09-15_262.png
659 883
Saved file: high-latitude-dust_2003-09-18_260.png
486 614
Saved f

Saved file: high-latitude-dust_2013-11-17_217.png
702 850
Saved file: high-latitude-dust_2014-10-28_215.png
1795 1188
Saved file: high-latitude-dust_2015-04-01_117.png
1117 417
Saved file: high-latitude-dust_2015-04-01_118.png
718 204
Saved file: high-latitude-dust_2015-04-01_119.png
1830 1143
Saved file: high-latitude-dust_2015-04-01_43.png
1422 416
Saved file: high-latitude-dust_2015-04-02_120.png
1445 456
Saved file: high-latitude-dust_2015-04-02_44.png
1452 329
Saved file: high-latitude-dust_2015-04-02_76.png
6158 2833
Saved file: high-latitude-dust_2015-04-24_73.png
5342 2536
Saved file: high-latitude-dust_2015-04-24_74.png
1156 1015
Saved file: high-latitude-dust_2015-05-20_121.png
841 475
Saved file: high-latitude-dust_2015-05-20_122.png
1189 1004
Saved file: high-latitude-dust_2015-05-20_45.png
661 277
Saved file: high-latitude-dust_2015-05-21_123.png
724 504
Saved file: high-latitude-dust_2015-05-21_46.png
1176 200
Saved file: high-latitude-dust_2015-05-22_124.png
544 87
Saved

In [37]:
nasalist = list(zip(dateonly,bbox))
print (nasalist[1:3])
print (len(nasalist))
#check
print (bbox[1:3])
print (dateonly[1:3])

[('2002-10-23', [-19.05626, 62.637483, -17.143616, 63.602063]), ('2003-01-19', [-67.575581, -45.764561, -63.257844, -44.841727])]
206
[[-19.05626, 62.637483, -17.143616, 63.602063], [-67.575581, -45.764561, -63.257844, -44.841727]]
['2002-10-23', '2003-01-19']


In [38]:
with open('nasa.txt', 'w') as f:
    for item in nasalist:
        f.write(str(item) +"\n")